In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
import pandas as pd
import numpy as np

2023-08-07 10:47:58.286347: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-07 10:47:58.314485: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-07 10:47:58.779363: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv('Churn_Modelling.csv')
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [3]:
# Checking for NaN values
df.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [4]:
# Removing data which is not required

df.pop('RowNumber')
df.pop('CustomerId')
df.pop('Surname')
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [5]:
# Encoding Geography and Gender
from sklearn.preprocessing import OrdinalEncoder

ord_enc = OrdinalEncoder()
df["Geography"] = ord_enc.fit_transform(df[["Geography"]])
df["Gender"] = ord_enc.fit_transform(df[["Gender"]])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0.0,0.0,42,2,0.00,1,1,1,101348.88,1
1,608,2.0,0.0,41,1,83807.86,1,0,1,112542.58,0
2,502,0.0,0.0,42,8,159660.80,3,1,0,113931.57,1
3,699,0.0,0.0,39,1,0.00,2,0,0,93826.63,0
4,850,2.0,0.0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,0.0,1.0,39,5,0.00,2,1,0,96270.64,0
9996,516,0.0,1.0,35,10,57369.61,1,1,1,101699.77,0
9997,709,0.0,0.0,36,7,0.00,1,0,1,42085.58,1
9998,772,1.0,1.0,42,3,75075.31,2,1,0,92888.52,1


In [6]:
# Normalizing data
df = df/df.max()
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0.728235,0.0,0.0,0.456522,0.2,0.000000,0.25,1.0,1.0,0.506763,1.0
1,0.715294,1.0,0.0,0.445652,0.1,0.334031,0.25,0.0,1.0,0.562734,0.0
2,0.590588,0.0,0.0,0.456522,0.8,0.636357,0.75,1.0,0.0,0.569679,1.0
3,0.822353,0.0,0.0,0.423913,0.1,0.000000,0.50,0.0,0.0,0.469151,0.0
4,1.000000,1.0,0.0,0.467391,0.2,0.500246,0.25,1.0,1.0,0.395435,0.0
...,...,...,...,...,...,...,...,...,...,...,...
9995,0.907059,0.0,1.0,0.423913,0.5,0.000000,0.50,1.0,0.0,0.481371,0.0
9996,0.607059,0.0,1.0,0.380435,1.0,0.228657,0.25,1.0,1.0,0.508518,0.0
9997,0.834118,0.0,0.0,0.391304,0.7,0.000000,0.25,0.0,1.0,0.210436,1.0
9998,0.908235,0.5,1.0,0.456522,0.3,0.299226,0.50,1.0,0.0,0.464460,1.0


In [7]:
# Target column
def to_int(x):
    return int(x)
target = df['Exited']
target = target.apply(to_int)
target

0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

In [8]:
# Removing target from input
df.pop('Exited')

0       1.0
1       0.0
2       1.0
3       0.0
4       0.0
       ... 
9995    0.0
9996    0.0
9997    1.0
9998    1.0
9999    0.0
Name: Exited, Length: 10000, dtype: float64

In [9]:
# splitting train test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, target, test_size=0.15, random_state=42)

In [19]:
# Converting to tensors
input_train = tf.convert_to_tensor(X_train)
input_test = tf.convert_to_tensor(X_test)
output_train = tf.convert_to_tensor(y_train)
output_test = tf.convert_to_tensor(y_test)
output_test

<tf.Tensor: shape=(1500,), dtype=int64, numpy=array([0, 0, 0, ..., 0, 1, 0])>

In [11]:
num_features = len(input_train[0])
num_features

10

In [33]:
v = [((128,0,0),10,0.1),((128,64,0),20,0.01),((128,64,32),30,0.001)]
from sklearn.metrics import classification_report

def create_model(params):
    return tf.keras.Sequential([
            Input(shape=(10,)),
            tf.keras.layers.Dense(params[0], activation='relu'),
            tf.keras.layers.Dense(params[1], activation='relu'),
            tf.keras.layers.Dense(params[2],activation='relu'),
            tf.keras.layers.Dense(1,activation='sigmoid')
          ])

for value in v:
    print("MODEL 1")
    print(value[0])
    model = create_model(value[0])
    print(model.summary())
    epochs = value[1]
    lr = value[2]
    
    adam = tf.keras.optimizers.Adam(learning_rate=lr)

    model.compile(optimizer=adam,loss=tf.keras.losses.BinaryCrossentropy(), metrics='accuracy')
    
    batch_size = 32
    model.fit(input_train, output_train, epochs=epochs,validation_split=0.1)

    loss, accuracy = model.evaluate(input_test, output_test)

    print("LOSS: ",loss)
    print("ACCURACY", accuracy)

    output = model.predict(input_test)
    print(classification_report(output_test.numpy().tolist(),np.round(np.squeeze(output))))

MODEL 1
(128, 0, 0)
Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_60 (Dense)            (None, 128)               1408      
                                                                 
 dense_61 (Dense)            (None, 0)                 0         
                                                                 
 dense_62 (Dense)            (None, 0)                 0         
                                                                 
 dense_63 (Dense)            (None, 1)                 1         
                                                                 
Total params: 1409 (5.50 KB)
Trainable params: 1409 (5.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/10
240/240 [==============================] - 2s 4ms/step - loss: 0.5134 - accuracy: 0.7941 - val_loss: 0.5023 - val_acc

/home/mca/anaconda3/envs/dse/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mca/anaconda3/envs/dse/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mca/anaconda3/envs/dse/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


240/240 [==============================] - 2s 5ms/step - loss: 0.5546 - accuracy: 0.7941 - val_loss: 0.5015 - val_accuracy: 0.8012
Epoch 2/20
240/240 [==============================] - 1s 4ms/step - loss: 0.5087 - accuracy: 0.7941 - val_loss: 0.4988 - val_accuracy: 0.8012
Epoch 3/20
240/240 [==============================] - 1s 5ms/step - loss: 0.5086 - accuracy: 0.7941 - val_loss: 0.4988 - val_accuracy: 0.8012
Epoch 4/20
240/240 [==============================] - 1s 6ms/step - loss: 0.5086 - accuracy: 0.7941 - val_loss: 0.4994 - val_accuracy: 0.8012
Epoch 5/20
240/240 [==============================] - 1s 5ms/step - loss: 0.5087 - accuracy: 0.7941 - val_loss: 0.4992 - val_accuracy: 0.8012
Epoch 6/20
240/240 [==============================] - 1s 5ms/step - loss: 0.5086 - accuracy: 0.7941 - val_loss: 0.4991 - val_accuracy: 0.8012
Epoch 7/20
240/240 [==============================] - 1s 5ms/step - loss: 0.5088 - accuracy: 0.7941 - val_loss: 0.4993 - val_accuracy: 0.8012
Epoch 8/20
240/24

/home/mca/anaconda3/envs/dse/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mca/anaconda3/envs/dse/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mca/anaconda3/envs/dse/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


240/240 [==============================] - 2s 5ms/step - loss: 0.4867 - accuracy: 0.7957 - val_loss: 0.4317 - val_accuracy: 0.8118
Epoch 2/30
240/240 [==============================] - 1s 5ms/step - loss: 0.4492 - accuracy: 0.8098 - val_loss: 0.4549 - val_accuracy: 0.8024
Epoch 3/30
240/240 [==============================] - 1s 5ms/step - loss: 0.4301 - accuracy: 0.8166 - val_loss: 0.4068 - val_accuracy: 0.8376
Epoch 4/30
240/240 [==============================] - 1s 5ms/step - loss: 0.4141 - accuracy: 0.8306 - val_loss: 0.3956 - val_accuracy: 0.8435
Epoch 5/30
240/240 [==============================] - 1s 5ms/step - loss: 0.3964 - accuracy: 0.8371 - val_loss: 0.3762 - val_accuracy: 0.8506
Epoch 6/30
240/240 [==============================] - 1s 5ms/step - loss: 0.3793 - accuracy: 0.8468 - val_loss: 0.3861 - val_accuracy: 0.8353
Epoch 7/30
240/240 [==============================] - 1s 5ms/step - loss: 0.3598 - accuracy: 0.8516 - val_loss: 0.3528 - val_accuracy: 0.8506
Epoch 8/30
240/24

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
